<a href="https://colab.research.google.com/github/ucfilho/raianars_fly/blob/master/RTC_DE_fev_21_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
#FUN calcula o TAC para um determinado arranjo

def FUN(x):
  global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  #latenteH2O=2256 #[kJ/kg] (vap sat, 2 bar)
  #cpH2O=4.1868 #[kJ/(kg.K)]
  # print(mCp)
  # rows,cols=mCp.shape 
  cols=len(mCp)
  matrizDeltaT=np.zeros((cols,cols))
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizDeltaT[i,j]=x[contador]
        contador=contador+1
  contador=0
  matrizBinaria=np.zeros((cols,cols))
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizBinaria[i,j]=round(x[contador+6])
        contador=contador+1
    
  areaTrocador=0;tempSaidaQ=0;tempSaidaF=0;deltaTMediaLog=0;
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        if(matrizBinaria[i,j]==1):
          if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
            #i: CQ, j: CF
            cargaTermica=mCp[i]*matrizDeltaT[i,j]
            tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
            if(
                tempAtual[i]>tempAtual[j] and 
                tempSaidaQ>tempAtual[j] and 
                tempAtual[i]>tempSaidaF and
                contador < cols
              ):
              #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
              
              #deltaTMediaLog=A/B
              try:
                A=((tempSaidaQ-tempSaidaF)-(tempAtual[i]-tempAtual[j]))
                B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[i]-tempAtual[j]))
                deltaTMediaLog=A/B
              except:
                deltaTMediaLog=1e99
              areaTrocador=areaTrocador+abs(cargaTermica/(U*deltaTMediaLog))
              tempAtual[i]=tempSaidaQ
              tempAtual[j]=tempSaidaF
          else:#a corrente i ganha calor da j
            #i: CF, j: CQ
            cargaTermica=mCp[i]*matrizDeltaT[i,j]
            tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
            if(
                  
                 tempAtual[i]< tempAtual[j] and
                 tempSaidaF<tempAtual[j] and
                 tempAtual[i]<tempSaidaQ and 
                 contador < cols

              ):#TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
              
              #deltaTMediaLog=A/B
              try:
                A=((tempSaidaQ-tempSaidaF)-(tempAtual[j]-tempAtual[i]))
                B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[j]-tempAtual[i]))
                deltaTMediaLog=A/B
              except:
                deltaTMediaLog=1e99

              areaTrocador=areaTrocador+abs(cargaTermica/(U*deltaTMediaLog))
              tempAtual[i]=tempSaidaF
              tempAtual[j]=tempSaidaQ
                            
                      
   
  cargaUQ=0; cargaUF=0;
  for i in range(cols):
    deltaT=tempAlvo[i]-tempAtual[i]
    if(deltaT>0):#usa UQ
      cargaUQ=cargaUQ+mCp[i]*deltaT
      #UQ -> vapor saturado -> entra e sai a 100ºC
      #A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
      #B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
      # deltaTMediaLog=A/B
      try:
        A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
        B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      areaTrocador=areaTrocador+abs(mCp[i]*(deltaT)/(U*deltaTMediaLog))
    else:#usa UF      
      cargaUF=cargaUF+mCp[i]*(-deltaT)
      #UF -> água resfriada -> entra a 5ºC e sai a 20ºC
      try:
        A=((tempAlvo[i]-293.15)-(tempAtual[i]-278.15))
        B=np.log((tempAlvo[i]-293.15)/(tempAtual[i]-278.15))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      areaTrocador=areaTrocador+abs(mCp[i]*(-deltaT)/(U*deltaTMediaLog))
  try:
    costTrocador=300*(areaTrocador)**0.5
  except:
    costTrocador=1e99
  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF
  funr=costUQ+costUF+costTrocador
  
  return funr

In [0]:
#MATGRAF retorna o gráfico das trocas térmicas em cada trocador
# function matrizGrafico=MATGRAF(matrizDeltaT,mCp,tempAtual)
def MATGRAF(matrizDeltaT,mCp,tempAtual):
  cols=len(mCp)
  matrizGrafico=np.zeros((cols,cols))
  contador=0
  matrizGrafico[:,0]=tempAtual
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
          #i: CQ, j:CF
          cargaTermica=mCp[i]*matrizDeltaT[i,j]
          tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
          tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
          if(
              tempAtual[i]>tempAtual[j] and
              tempSaidaQ>tempAtual[j] and
              tempAtual[i]>tempSaidaF and
              contador < cols
              
            ):
            tempAtual[i]=tempSaidaQ
            tempAtual[j]=tempSaidaF
            matrizGrafico[:,contador]=tempAtual
            
            #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
            contador=contador+1
            # troquei o elif por else (fica melhor no contexto?)
            # elif (matrizDeltaT[i,j]>0) : #a corrente i ganha calor da j
      else : #a corrente i ganha calor da j

        # situa
        cargaTermica=mCp[i]*matrizDeltaT[i,j]
        tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
        tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j] 
        if (
          
            tempAtual[i]<tempAtual[j] and 
            tempSaidaF<tempAtual[j] and 
            tempAtual[i]<tempSaidaQ and
            contador< cols

            ):
            #  i: CF, j: CQ
              
            tempAtual[i]=tempSaidaF
            tempAtual[j]=tempSaidaQ
            matrizGrafico[:,contador]=tempAtual
            contador=contador+1          

  return matrizGrafico

In [0]:
#MATBEST retorna a matriz com os deltas das trocas térmicas que ocorreram e a matriz binária
# function [matrizTrocaBest,matrizBinaria]=MATBEST(x,mCp,tempAtual)
def MATBEST(x,mCp,tempAtual):
  
  nref=int(len(x)/2)
  #nref=0
  cols=len(mCp)

  # cada particula contem metade das variaveis para matriz binaria
  # e outra metade para deltaT nref e para separar os dois casos
  matrizDeltaT=np.zeros((cols,cols))
  matrizTrocaBest=np.zeros((cols,cols))
  # contador=-1
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        # contador=contador+1
        matrizDeltaT[i,j]=x[contador]
        contador=contador+1
    
    '''
    nao tem esta linha (contador=0) abaixo 
    no original mas se tirar ela gera out of bounds
    '''
    contador=0 
    matrizBinaria=np.zeros((cols,cols))
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          # matrizBinaria[i,j]=round(x[contador-1])
          '''
          parece q faz mais sentido assim (forma abaixo)
          '''
          matrizBinaria[i,j]=round(x[contador+nref])
          # print('matriz binaria',matrizBinaria[i,j])
          contador=contador+1
     
    tempSaidaQ=0;tempSaidaF=0;cargaTermica=0
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          if(matrizBinaria[i,j]==1):
            if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j 
              #i: CQ, j: CF
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]>tempAtual[j] and
                  tempSaidaQ>tempAtual[j] and
                  tempAtual[i]>tempSaidaF
                ): #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
                tempAtual[i]=tempSaidaQ
                tempAtual[j]=tempSaidaF
              else:
                matrizDeltaT[i,j]=0
            else:#a corrente i ganha calor da j
              # i: CF, j:CQ
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]<tempAtual[j] and
                  tempSaidaF<tempAtual[j] and
                  tempAtual[i]<tempSaidaQ
                 ):
                #TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
                tempAtual[i]=tempSaidaF
                tempAtual[j]=tempSaidaQ
              else:
                matrizDeltaT[i,j]=0
          else:
            matrizDeltaT[i,j]=0
  print('contador=',contador)
  matrizTrocaBest=matrizDeltaT
  return matrizTrocaBest,matrizBinaria

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=50 #Número de partículas
ITE=100 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[100,100,100,100,100,100,1,1,1,1,1,1]
MIN=[-100,-100,-100,-100,-100,-100,0,0,0,0,0,0]

mCp=[4,2,1.5,3] #[kW/K]
tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]
coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

x=[ 74,20,83,-10,38,-50,0,0,0,0,1,1]
resp=FUN(x)
print('===++++++++++======')
print(resp)

x=[ 90,20,83,-10,38,-50,1,1,0,0,1,1]
resp=FUN(x)
print('==xxxxxxxxxx=====')
print(resp)

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))

#print(bounds)
print('temperatura atual=',tempAtual)
print('===========================')
print('===========================')
print('===========================')
fchoice=FUN
result = differential_evolution(fchoice, bounds, maxiter=ITE, popsize=NPAR)
print('temperatura atual=',tempAtual)
print(result.x, result.fun)
print(tempAtual)
GBEST=result.x
BEST=result.fun
matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n',ITE,NPAR);print(GBEST)
print('Valor da função f(gbest)=%f \n',BEST)

print('Matriz de Trocas Térmicas \n')
print(matrizTrocaBest)
print('Matriz Binária \n')
print(matrizBinaria)


===++++++++++======
59535.52238221133
==xxxxxxxxxx=====
59535.52238221133
temperatura atual= [453, 393, 523, 533]
temperatura atual= [453, 393, 523, 533]
[65.43154418 39.42819381 86.80727033  4.87762284 91.30366972 58.63499452
  0.15191189  0.5110957   0.54020704  0.77881369  0.70661464  0.88714572] 59535.52238221133
[453, 393, 523, 533]
contador= 6
Melhor solução para %d iterações com %d partículas 
 100 50
[65.43154418 39.42819381 86.80727033  4.87762284 91.30366972 58.63499452
  0.15191189  0.5110957   0.54020704  0.77881369  0.70661464  0.88714572]
Valor da função f(gbest)=%f 
 59535.52238221133
Matriz de Trocas Térmicas 

[[0.         0.         0.         0.        ]
 [0.         0.         0.15191189 0.5110957 ]
 [0.         0.         0.         0.15191189]
 [0.         0.         0.         0.        ]]
Matriz Binária 

[[0. 0. 1. 1.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
